In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import cv2
import random
import pickle
import itertools 
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
#from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import optimizers
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix

%matplotlib inline

2025-03-14 16:21:32.687530: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-14 16:21:32.693092: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 16:21:32.743588: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 16:21:32.790891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741936892.831970   38183 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741936892.84

## 함수 정의 

In [2]:
def plot_results(mod_history, metric, epochs):
      
      # Check out our train loss and test loss over epochs.
      train_metric = mod_history.history[metric]
      val = 'val_' + metric
      test_metric = mod_history.history[val]

      # Set figure size.
      plt.figure(figsize=(12, 8))

      # Generate line plot of training, testing loss over epochs.
      plt.plot(train_metric, label=f'Training {metric}', color='#185fad')
      plt.plot(test_metric, label=f'Testing {metric}', color='orange')

      # Set title
      plt.title(f'Training and Testing {metric} by Epoch', fontsize = 25)
      plt.xlabel('Epoch', fontsize = 18)
      plt.ylabel('Categorical Crossentropy', fontsize = 18)
      plt.xticks(range(0,epochs,5), range(0,epochs,5))
      plt.legend(fontsize = 18);

In [3]:
def make_predictions(mod_name, steps=20):
    preds = mod_name.predict(X_test,steps=steps)
    preds = preds.argmax(axis=-1)

    y_test_labels = np.argmax(y_test, axis=-1)

    cm = confusion_matrix(y_test_labels,preds)

    plot_confusion_matrix(cm, cm_plot_labels, normalize=True,
                          title='Face Shape Normalized')

    plt.show()

In [4]:
cm_plot_labels = ['Heart','Oblong','Oval','Round', 'Square']

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.figure(figsize=(16,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [5]:
def plot_summary_results(mod_name, mod_history, epochs):
    plot_results(mod_history, 'loss',epochs)
    plot_results(mod_history, 'accuracy', epochs)
    make_predictions(mod_name)

In [2]:
path = "./face_shape_classification/"

X_train = np.asarray(pickle.load(open(path + "X_train_gray.pickle","rb")))
y_train = np.asarray(pickle.load(open(path + "y_train_gray.pickle","rb")))
X_test = np.asarray(pickle.load(open(path + "X_test_gray.pickle","rb")))
y_test = np.asarray(pickle.load(open(path + "y_test_gray.pickle","rb")))

In [3]:
print("Data Summary")
print("--------------------")
print(f"X_train shape {X_train.shape}")
print(f"y_train shape {y_train.shape}")
print("--------------------")
print(f"X_test shape {X_test.shape}")
print(f"y_test shape {y_test.shape}")

Data Summary
--------------------
X_train shape (3981, 224, 224, 1)
y_train shape (3981, 5)
--------------------
X_test shape (998, 224, 224, 1)
y_test shape (998, 5)


## VGG16 모델 활용한 전이학습

In [7]:
import tensorflow as tf

# GPU 사용을 비활성화하여 CPU에서만 실행
tf.config.set_visible_devices([], 'GPU')

In [4]:
path_vggface = './saved_face_models/rcmalli_vggface_tf_notop_vgg16.h5'

In [5]:
#흑백 이미지 3채널로 변환하기 
import numpy as np

# X_train, X_test의 shape가 (N, 224, 224, 1)일 때, 3채널로 변환
X_train_rgb = np.repeat(X_train, 3, axis=-1)  # 1채널을 3채널로 확장
X_test_rgb = np.repeat(X_test, 3, axis=-1)    # 1채널을 3채널로 확장

In [8]:
base_model = VGG16(input_shape=(224, 224, 3),  # same as our input
                   include_top=False,  # exclude the last layer
                   weights=path_vggface)  # use VGGFace Weights

In [9]:
for layer in base_model.layers:
  layer.trainable = False

In [10]:
model_t1 = Sequential()

In [11]:
x = layers.Flatten()(base_model.output)

x = layers.Dense(64, activation='relu')(x)  # add 1 fully connected layer, try with 512 first 
x = layers.Dropout(0.5)(x)
x = layers.Dense(5, activation='softmax')(x)  # add final layer

model_t1 = tf.keras.models.Model(base_model.input, x)



model_t1.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

model_t1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,320,709 (62.26 MB)

 Trainable params: 1,606,021 (6.13 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [13]:
datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True)

In [14]:
datagen.fit(X_train_rgb)

In [16]:
batch_size = 64
steps_per_epoch = len(X_train_rgb) // batch_size

history_t1 = model_t1.fit(datagen.flow(X_train_rgb, y_train, batch_size=batch_size), 
                          steps_per_epoch=len(X_train) // batch_size, epochs=10, 
                          validation_data=(X_test_rgb, y_test))  

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 377s 6s/step - accuracy: 0.7019 - loss: 0.8329 - val_accuracy: 0.8267 - val_loss: 0.5539
Epoch 2/10
 1/62 ━━━━━━━━━━━━━━━━━━━━ 5:07 5s/step - accuracy: 0.8438 - loss: 0.6619

/home/encore/tensflow/venv/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


62/62 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.8438 - loss: 0.6619 - val_accuracy: 0.8267 - val_loss: 0.5500
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 390s 6s/step - accuracy: 0.7617 - loss: 0.6759 - val_accuracy: 0.8407 - val_loss: 0.4764
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.7500 - loss: 0.6785 - val_accuracy: 0.8417 - val_loss: 0.4768
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 390s 6s/step - accuracy: 0.8056 - loss: 0.5750 - val_accuracy: 0.8507 - val_loss: 0.4218
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.7500 - loss: 0.6377 - val_accuracy: 0.8487 - val_loss: 0.4226
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 370s 6s/step - accuracy: 0.8232 - loss: 0.5100 - val_accuracy: 0.8798 - val_loss: 0.3856
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.8438 - loss: 0.4762 - val_accuracy: 0.8778 - val_loss: 0.3850
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 378s 6s/step - accuracy: 0.8438 - loss: 0.4405 - val_accuracy: 0.8858 - val_loss: 0.3481

In [17]:
max(history_t1.history['val_accuracy'])

0.8857715725898743

In [19]:
model_t1.evaluate(X_test_rgb, y_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.8875 - loss: 0.3595


[0.348457396030426, 0.8837675452232361]

In [21]:
model_path = './saved_face_models/'
filename = model_path + 'vgg16.keras'

#tf.keras.models.save_model( 
#   model, filepath=model_path, overwrite=True, include_optimizer=True, save_format=None,
#   signatures='some_signatures', options='some_options')
 
model_t1.save(filename, overwrite=True, include_optimizer=True)